In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import os
import glob
from keras.models import Sequential
from keras.layers import InputLayer, Dense, Conv2D, MaxPool2D, Flatten

%matplotlib inline

In [4]:
data_dir = 'data/miniimagenet/'
train_dir = os.path.join(data_dir, 'train')
test_dir = os.path.join(data_dir, 'train')
valid_dir = os.path.join(data_dir, 'train')

In [5]:
IMG_SIZE = 64
IMG_CHANNELS = 3
N_CLASSES = len(os.listdir(train_dir))
N_INNER_CLASSES = 10
INNER_EPOCHS = 1
INNER_BATCH_SIZE = 64
N_INNER_STEPS = 10
INNER_LR = 0.001
OUTER_EPOCHS = 1000
OUTER_LR = 0.01

## Define Image Loading Functions

In [6]:
img_cache = {}

In [7]:
def preprocess_img(img, size=IMG_SIZE):
    return cv2.resize(img, (size, size))

def load_img(class_dir, img_name, img_path=None):
    if img_path:
        img_name = img_path[img_path.rfind('/')+1:]
    else:
        img_path = os.path.join(class_dir, img_name[:img_name.find('_')], img_name)
    
    # If image is already loaded into cache return it
    if img_name in img_cache:
        return img_cache[img_name]
    else:
        img = cv2.imread(img_path)
        img = preprocess_img(img, size=IMG_SIZE)
        img_cache[img_name] = img
        return img

## Building the DataFrame of Image Names

In [8]:
train_classes = np.asarray(os.listdir(train_dir))

In [9]:
cols = ['class', 'img_name']
df_train = pd.DataFrame(columns=cols)

for cat in train_classes:
    class_list = []
    cat_dir = os.listdir(os.path.join(train_dir, cat))
    for img_name in cat_dir:
        class_list.append([cat, img_name])
    tmp_df = pd.DataFrame(class_list, columns=cols)
    df_train = df_train.append(tmp_df)
    
df_train = df_train.sample(frac=1)
df_train.reset_index(inplace=True)
df_train.drop('index', inplace=True, axis=1)

## Functions for Generating Tasks and Formatting Data

In [10]:
def sample_task(data, n_classes=N_INNER_CLASSES):
    classes = np.random.choice(train_classes, size=n_classes, replace=False)
    task_indices = data['class'].map(lambda x: x in classes)
    task_data = data[task_indices]
    return task_data

def gen_batches(task_data, batch_size=INNER_BATCH_SIZE, data_dir=train_dir):
    task_data = pd.concat([task_data, pd.get_dummies(task_data['class'])], axis=1)
    while True:
        epoch_data = task_data.sample(frac=1)
        for i in range((len(epoch_data) // batch_size) - 1):
            print(i)
            X_names = epoch_data.iloc[i*batch_size:(i+1)*batch_size]['img_name'].values
            X = np.asarray([load_img(data_dir, img_name) for img_name in X_names])
            y = epoch_data.drop(['class', 'img_name'], axis=1).iloc[i*batch_size:(i+1)*batch_size].values
            
            yield X, y

## Building a Model

In [11]:
model = Sequential()
model.add(InputLayer(input_shape=(IMG_SIZE, IMG_SIZE, IMG_CHANNELS)))
model.add(Conv2D(32, 5, activation='relu'))
model.add(MaxPool2D())
model.add(Conv2D(64, 3, activation='relu'))
model.add(Conv2D(64, 3, activation='relu'))
model.add(Flatten())
model.add(Dense(N_INNER_CLASSES, activation='softmax'))

model.compile(optimizer='SGD',
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 60, 60, 32)        2432      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 30, 30, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 28, 28, 64)        18496     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 26, 26, 64)        36928     
_________________________________________________________________
flatten_1 (Flatten)          (None, 43264)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                432650    
Total params: 490,506
Trainable params: 490,506
Non-trainable params: 0
_________________________________________________________________


## Training the Model

In [15]:
for i in range(OUTER_EPOCHS):
    print('------', len(img_cache))
    task = sample_task(df_train)
    print(task.head())
    data = gen_batches(task)
    initial_weights = model.get_weights()
    model.fit_generator(data,
              steps_per_epoch=N_INNER_STEPS,
              epochs=1, 
              verbose=1)
    model.set_weights([iw + OUTER_LR * (cw - iw)
                       for iw, cw in zip(initial_weights, model.get_weights())])

------ 13887
        class              img_name
0   n04509417     n04509417_30.JPEG
9   n04067472   n04067472_2244.JPEG
16  n02108915   n02108915_8181.JPEG
26  n03047690  n03047690_15969.JPEG
40  n02457408  n02457408_10004.JPEG
Epoch 1/1
0
1
10/10 [==============================] - 7s 677ms/step - loss: 14.3698 - acc: 0.1078
------ 14314
        class              img_name
2   n01704323   n01704323_1152.JPEG
7   n04604644   n04604644_8829.JPEG
9   n04067472   n04067472_2244.JPEG
13  n04251144  n04251144_19600.JPEG
23  n04604644   n04604644_9379.JPEG
Epoch 1/1
0
1
10/10 [==============================] - 5s 548ms/step - loss: 14.7329 - acc: 0.0859
------ 14749
        class              img_name
0   n04509417     n04509417_30.JPEG
20  n02074367   n02074367_1985.JPEG
26  n03047690  n03047690_15969.JPEG
28  n02113712    n02113712_919.JPEG
30  n02165456  n02165456_16051.JPEG
Epoch 1/1
0
1
10/10 [==============================] - 6s 595ms/step - loss: 14.3048 - acc: 0.1125
------ 15178
   

10/10 [==============================] - 7s 686ms/step - loss: 15.0100 - acc: 0.0687
------ 17219
        class              img_name
11  n01843383   n01843383_7342.JPEG
13  n04251144  n04251144_19600.JPEG
21  n07747607  n07747607_20395.JPEG
30  n02165456  n02165456_16051.JPEG
32  n04443257  n04443257_12885.JPEG
Epoch 1/1
0
1
10/10 [==============================] - 5s 539ms/step - loss: 14.5818 - acc: 0.0953
------ 17576
        class              img_name
0   n04509417     n04509417_30.JPEG
3   n04296562  n04296562_16372.JPEG
10  n03337140  n03337140_39623.JPEG
29  n03337140    n03337140_740.JPEG
38  n04389033  n04389033_40682.JPEG
Epoch 1/1
0
1
10/10 [==============================] - 4s 411ms/step - loss: 14.3804 - acc: 0.1078
------ 17903
        class              img_name
5   n03924679  n03924679_20279.JPEG
8   n03908618  n03908618_18146.JPEG
19  n03924679   n03924679_6787.JPEG
20  n02074367   n02074367_1985.JPEG
26  n03047690  n03047690_15969.JPEG
Epoch 1/1
0
1
10/10 [=========

KeyboardInterrupt: 

In [325]:
np.asarray(model.get_weights())[7].shape

(10,)

In [326]:
model.get_weights() - model.get_weights()

TypeError: unsupported operand type(s) for -: 'list' and 'list'